<a href="https://colab.research.google.com/github/AmanpreetSingh-GitHub/Deep-Learning---Identifying-Spam-Questions/blob/main/Identifying%20Spam%20Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import required libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from nltk import word_tokenize
import nltk
nltk.download('punkt')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from tensorflow.keras.models import Model

from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Download the data from dropbox**

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2021-10-11 09:41:25--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2021-10-11 09:41:26--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc61c79043a2bc095df9442e8813.dl.dropboxusercontent.com/zip_download_get/A7Oko6gRXTTN1vBHBbGVWJyX0PrbtivfBftlLQ7HjFGA7M9Q1tBn1B9MiY_ua5sn0RJlpkB-cfwYimd1c26vhYyVtrKWbqCb6cFssw38NGaBog# [following]
--2021-10-11 09:41:26--  https://uc61c79043a2bc095df9442e8813.dl.dropboxusercontent.com/zip_download_get/A7Oko6gRXTTN1vBHBbGVWJyX0PrbtivfBftlLQ7HjFGA7M9Q1tBn1B9MiY_ua5sn0RJlpkB-cfwYimd1

**Read the data**

In [3]:
fs_data = pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0', sep = ',', encoding = 'latin-1')

In [4]:
fs_data

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


**Explore and Clean the data**

Removing NA columns

In [5]:
fs_data.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [6]:
fs_data.dropna(inplace=True)

In [7]:
fs_data.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

Renaming columns and removing unnecessary column PK

In [8]:
fs_data.columns = ['PK', 'question_text', 'target']

In [9]:
fs_data.drop(['PK'], axis = 1, inplace = True)

In [10]:
fs_data

,question_text,target
0,How did Quebec nationalists see their province...,0.0
1,"Do you have an adopted dog, how would you enco...",0.0
2,Why does velocity affect time? Does velocity a...,0.0
3,How did Otto von Guericke used the Magdeburg h...,0.0
4,Can I convert montra helicon D to a mountain b...,0.0
...,...,...
1306117,What other technical skills do you need as a c...,0.0
1306118,Does MS in ECE have good job prospects in USA ...,0.0
1306119,Is foam insulation toxic?,0.0
1306120,How can one start a research project based on ...,0.0


**Downloading GloVe: Global Vectors for Word Representation**

GloVe is an unsupervised learning algorithm for obtaining vector representations for words.

Download GloVe, unzip it and remove unnecessary temp files

In [11]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip /content/glove.42B.300d.zip 
!rm /content/glove.42B.300d.zip

--2021-10-11 09:42:45--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-10-11 09:42:45--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-10-11 09:42:46--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

Reading the GloVe embedding

In [12]:
embedding_word_coeff = {}

f = open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embedding_word_coeff[word] = coefs

f.close()

**Split the data into train and validation, and tokenize it**

In [13]:
data_train, data_test = train_test_split(fs_data, test_size = 0.2)
  
x_train = data_train['question_text']
y_train = data_train['target']
x_test = data_test['question_text']
y_test = data_test['target']

In [14]:
sentence_length = [len(word_tokenize(x)) for x in x_train]

In [15]:
print(np.percentile(sentence_length, 95))
print(np.percentile(sentence_length, 96))
print(np.percentile(sentence_length, 97))
print(np.percentile(sentence_length, 98))
print(np.percentile(sentence_length, 99))

30.0
32.0
35.0
39.0
44.0


In [16]:
maximum_sentence_length = 44

tk = Tokenizer(char_level = False, split = ' ')

tk.fit_on_texts(x_train)

sequence_train = tk.texts_to_sequences(x_train)
sequence_test = tk.texts_to_sequences(x_test)

vocabulary_size = len(tk.word_index)

sequence_train_matrix = sequence.pad_sequences(sequence_train, maxlen=maximum_sentence_length)
sequence_test_matrix = sequence.pad_sequences(sequence_test, maxlen = maximum_sentence_length)

Creating our own custom embedding matrix to bring down the size to 300. If there are specific words which are not present in pretrained embedding their weights will remain 0

In [18]:
custom_embedding_matrix = np.zeros((vocabulary_size + 1, 300))

for word, index in tk.word_index.items():
    embedding_vector = embedding_word_coeff.get(word)
    if embedding_vector is not None:
        custom_embedding_matrix[index] = embedding_vector

**Creating Long Short-Term Memory (LSTM) Model**

In [20]:
input_to_model = Input(name = 'text_input', shape = [maximum_sentence_length])

embedding = Embedding(vocabulary_size + 1, 
                      300, 
                      input_length = maximum_sentence_length, 
                      mask_zero = True, 
                      weights = [custom_embedding_matrix],
                      trainable = False)(input_to_model)

first_lstm_layer = LSTM(512)(embedding)
first_dropout = Dropout(0.2)(first_lstm_layer)

second_lstm_layer = Dense(256, activation='relu')(first_dropout)
second_dropout = Dropout(0.2)(second_lstm_layer)

third_lstm_layer = Dense(128, activation='relu')(second_dropout)
third_dropout = Dropout(0.2)(third_lstm_layer)

fourth_lstm_layer = Dense(25, activation='relu')(third_dropout)
fourth_dropout = Dropout(0.2)(fourth_lstm_layer)

output_layer = Dense(1, activation = 'sigmoid')(fourth_dropout)

lstmModel = Model(inputs = input_to_model, outputs = output_layer)

In [21]:
lstmModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58855800  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896 

In [22]:
lstmModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
lstmModel.fit(sequence_train_matrix,
          y_train,
          epochs = 14,
          batch_size = 800,
          validation_data = (sequence_test_matrix, 
                             y_test)
          )

Epoch 1/14
1307/1307 [==============================] - 409s 307ms/step - loss: 0.1215 - accuracy: 0.9532 - val_loss: 0.1088 - val_accuracy: 0.9569
Epoch 2/14
1307/1307 [==============================] - 401s 307ms/step - loss: 0.1054 - accuracy: 0.9586 - val_loss: 0.1076 - val_accuracy: 0.9583
Epoch 3/14
1307/1307 [==============================] - 399s 305ms/step - loss: 0.0987 - accuracy: 0.9608 - val_loss: 0.1044 - val_accuracy: 0.9564
Epoch 4/14
1307/1307 [==============================] - 399s 305ms/step - loss: 0.0922 - accuracy: 0.9629 - val_loss: 0.1010 - val_accuracy: 0.9601
Epoch 5/14
1307/1307 [==============================] - 400s 306ms/step - loss: 0.0848 - accuracy: 0.9652 - val_loss: 0.1029 - val_accuracy: 0.9597
Epoch 6/14
1307/1307 [==============================] - 402s 307ms/step - loss: 0.0757 - accuracy: 0.9682 - val_loss: 0.1120 - val_accuracy: 0.9593
Epoch 7/14
1307/1307 [==============================] - 401s 307ms/step - loss: 0.0653 - accuracy: 0.9723 - val_

**Predicting on test data and finding roc auc score**

In [24]:
prediction = lstmModel.predict(sequence_test_matrix)

In [25]:
roc_auc_score(y_test, prediction)

0.95155444343768